

# **Data Science, 2024**



A demonstration of exploratory data analysis to accompany the Lecture.

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

These options are used to customize the display of pandas DataFrames for better readability and precision when working with data.

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
# This option stops scientific notation for pandas
pd.set_option('display.float_format', '{:.2f}'.format)


# Tuberculosis in the United States

What can we say about the presence of Tuberculosis in the United States?

Let's look at the data included in the [original CDC article](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down) published in 2021.

**CSV and Nice Field Names**

Suppose Table 1 was saved as a CSV file located in `data/cdc_tuberculosis.csv`. We can then explore the CSV (which is a text
file, and does not contain binary-encoded data) in many ways:
1. Using a text editor like the one in DataHub (right-click on the file and use `Open->Editor`), emacs, vim, VSCode, etc.
2. Opening the CSV directly in DataHub (read-only), Excel, Google Sheets, etc.
3. The Python file object
4. pandas, using `pd.read_csv()`

Let's start with the first two so we solidify the idea of a CSV as **rectangular data (i.e., tabular data) stored as comma-separated values**.

Try 1, 2.

Then Let's see the tried-and-true Data Science approach: pandas.

In [3]:
# Read tuberculosis data
tuberculosis=pd.read_csv('./data/cdc_tuberculosis.csv')
tuberculosis

,Unnamed: 0,No. of TB cases,Unnamed: 2,Unnamed: 3,TB incidence,Unnamed: 5,Unnamed: 6
0,U.S. jurisdiction,2019,2020,2021,2019.00,2020.00,2021.00
1,Total,"8,900","7,173","7,860",2.71,2.16,2.37
2,Alabama,87,72,92,1.77,1.43,1.83
3,Alaska,58,58,58,7.91,7.92,7.92
4,Arizona,183,136,129,2.51,1.89,1.77
...,...,...,...,...,...,...,...
48,Virginia,191,169,161,2.23,1.96,1.86
49,Washington,221,163,199,2.90,2.11,2.57
50,West Virginia,9,13,7,0.50,0.73,0.39
51,Wisconsin,51,35,66,0.88,0.59,1.12


Wait, what's up with the "Unnamed" column names? And the first row, for that matter?

Congratulations -- you're ready to wrangle your data. Because of how things are stored, we'll need to clean the data a bit to name our columns better.

A reasonable first step is to identify the row with the right header. The `pd.read_csv()` function ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) has the convenient `header` parameter:

In [4]:
# Answer Here
tuberculosis=pd.read_csv('./data/cdc_tuberculosis.csv',header=1)
tuberculosis

,U.S. jurisdiction,2019,2020,2021,2019.1,2020.1,2021.1
0,Total,"8,900","7,173","7,860",2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


Wait...but now we can't differentiate betwen the "Number of TB cases" and "TB incidence" year columns. pandas has tried to make our lives easier by automatically adding ".1" to the latter columns, but this doesn't help us as humans understand the data.

We can do this manually with `df.rename()`

In [5]:
# Rename column List
tuberculosis.columns=['U.S. jurisdiction','TB cases 2019','TB cases 2020','TB cases 2021',
                      'TB incidence 2019','TB incidence 2020','TB incidence 2021'
                      ]
tuberculosis

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
0,Total,"8,900","7,173","7,860",2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


# Record Granularity

You might already be wondering: What's up with that first record?

Row 0 is what we call a **rollup record**, or summary record. It's often useful when displaying tables to humans. The **granularity** of record 0 (Totals) vs the rest of the records (States) is different.


Okay, EDA step two. How was the rollup record aggregated?

Let's check if Total TB cases is the sum of all state TB cases. If we sum over all rows, we should get **2x** the total cases in each of our TB cases by year (why?).

In [6]:
# Apply sum to tb DataFrame
tuberculosis.sum()

U.S. jurisdiction    TotalAlabamaAlaskaArizonaArkansasCaliforniaCol...
TB cases 2019        8,9008758183642,111666718245583029973261085237...
TB cases 2020        7,1737258136591,706525417194122219282169239376...
TB cases 2021        7,8609258129691,750585443194992281064255127494...
TB incidence 2019                                               109.94
TB incidence 2020                                                93.09
TB incidence 2021                                               102.94
dtype: object


Whoa, what's going on? Check out the column types:

In [7]:
# Find datatype for each column
tuberculosis.dtypes

U.S. jurisdiction     object
TB cases 2019         object
TB cases 2020         object
TB cases 2021         object
TB incidence 2019    float64
TB incidence 2020    float64
TB incidence 2021    float64
dtype: object

Looks like those commas are causing all TB cases to be read as the `object` datatype, or **storage type** (close to the Python string datatype), so pandas is concatenating strings instead of adding integers.

Fortunately `read_csv` also has a `thousands` parameter (for what it's worth, I didn't know this beforehand--I [googled](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) this):

In [8]:
# remove the , sign using lambda function
tuberculosis['TB cases 2019']=tuberculosis['TB cases 2019'].apply(lambda x:x.replace(',','') if ',' in str(x) else x)
tuberculosis['TB cases 2020']=tuberculosis['TB cases 2020'].apply(lambda x:x.replace(',','') if ',' in str(x) else x)
tuberculosis['TB cases 2021']=tuberculosis['TB cases 2021'].apply(lambda x:x.replace(',','') if ',' in str(x) else x)
# convert them into integer using lambda function
tuberculosis['TB cases 2019']=tuberculosis['TB cases 2019'].apply(lambda x: int(x))
tuberculosis['TB cases 2020']=tuberculosis['TB cases 2020'].apply(lambda x: int(x))
tuberculosis['TB cases 2021']=tuberculosis['TB cases 2021'].apply(lambda x: int(x))

tuberculosis

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


In [9]:
# now apply sum
# Answer Here
tuberculosis.sum()

U.S. jurisdiction    TotalAlabamaAlaskaArizonaArkansasCaliforniaCol...
TB cases 2019                                                    17800
TB cases 2020                                                    14346
TB cases 2021                                                    15720
TB incidence 2019                                               109.94
TB incidence 2020                                                93.09
TB incidence 2021                                               102.94
dtype: object

The Total TB cases look right. Phew!

(We'll leave it to your own EDA to figure out how the TB incidence "Totals" were aggregated.)

Let's just look at the records with **state-level granularity**:

In [10]:
# Answer Here
tuberculosis_df=tuberculosis.iloc[1:,0:]
tuberculosis_df

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
5,California,2111,1706,1750,5.35,4.32,4.46
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


What do each of these values represent? Why?

To the lecture!


# Gather Census Data

U.S. Census population estimates [source](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html) (2019), [source](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html) (2020-2021).

Running the below cells cleans the data. We encourage you to closely explore the CSV and study these lines after lecture...

There are a few new methods here:
* `df.convert_dtypes()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.convert_dtypes.html)) conveniently converts all float dtypes into ints and is out of scope for the class.
* `df.drop_na()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)) will be explained in more detail next time.

In [11]:
# Load 2010s census data
# Answer
census = pd.read_csv('./data/st-est2019-01.csv',header=3)
census

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523"
1,Northeast,"55,317,240","55,318,443","55,380,134","55,604,223","55,775,216","55,901,806","56,006,011","56,034,684","56,042,330","56,059,240","56,046,620","55,982,803"
2,Midwest,"66,927,001","66,929,725","66,974,416","67,157,800","67,336,743","67,560,379","67,745,167","67,860,583","67,987,540","68,126,781","68,236,628","68,329,004"
3,South,"114,555,744","114,563,030","114,866,680","116,006,522","117,241,208","118,364,400","119,624,037","120,997,341","122,351,760","123,542,189","124,569,433","125,580,448"
4,West,"71,945,553","71,946,907","72,100,436","72,788,329","73,477,823","74,167,130","74,925,793","75,742,555","76,559,681","77,257,329","77,834,820","78,347,268"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Note: The estimates are based on the 2010 Cens...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Table 1. Annual Estimates of the Resident Popu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Apply some EDA

Drop the column of Estiamte Base

Rename Unnamed: 0 as 'Geographic Area'

"smart" converting of columns(.convert_dtypes()) Google it. Use at your own risk

.dropna() to drop records with NaN

You can also suggest any change that can be helpful for EDA



In [12]:
census = census.drop(columns='Estimates Base', axis=1)
census

,Unnamed: 0,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,"308,745,538","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523"
1,Northeast,"55,317,240","55,380,134","55,604,223","55,775,216","55,901,806","56,006,011","56,034,684","56,042,330","56,059,240","56,046,620","55,982,803"
2,Midwest,"66,927,001","66,974,416","67,157,800","67,336,743","67,560,379","67,745,167","67,860,583","67,987,540","68,126,781","68,236,628","68,329,004"
3,South,"114,555,744","114,866,680","116,006,522","117,241,208","118,364,400","119,624,037","120,997,341","122,351,760","123,542,189","124,569,433","125,580,448"
4,West,"71,945,553","72,100,436","72,788,329","73,477,823","74,167,130","74,925,793","75,742,555","76,559,681","77,257,329","77,834,820","78,347,268"
...,...,...,...,...,...,...,...,...,...,...,...,...
58,Note: The estimates are based on the 2010 Cens...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Table 1. Annual Estimates of the Resident Popu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
census.rename(columns={'Unnamed: 0':'Geographic Area'},inplace=True)

In [14]:
census.dropna(inplace=True)

In [15]:
census = census.apply(lambda col: col.astype(str).str.replace(',', '').apply(pd.to_numeric, errors='ignore'))
census

,Geographic Area,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
1,Northeast,55317240,55380134,55604223,55775216,55901806,56006011,56034684,56042330,56059240,56046620,55982803
2,Midwest,66927001,66974416,67157800,67336743,67560379,67745167,67860583,67987540,68126781,68236628,68329004
3,South,114555744,114866680,116006522,117241208,118364400,119624037,120997341,122351760,123542189,124569433,125580448
4,West,71945553,72100436,72788329,73477823,74167130,74925793,75742555,76559681,77257329,77834820,78347268
...,...,...,...,...,...,...,...,...,...,...,...,...
52,.Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893
53,.West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147
54,.Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434
55,.Wyoming,563626,564487,567299,576305,582122,582531,585613,584215,578931,577601,578759


In [16]:
# census 2020s data
census_20=pd.read_csv('./data/NST-EST2022-POP.csv',header=3)
census_20

,Unnamed: 0,Unnamed: 1,2020,2021,2022
0,United States,"331,449,520","331,511,512","332,031,554","333,287,557"
1,Northeast,"57,609,156","57,448,898","57,259,257","57,040,406"
2,Midwest,"68,985,537","68,961,043","68,836,505","68,787,595"
3,South,"126,266,262","126,450,613","127,346,029","128,716,192"
4,West,"78,588,565","78,650,958","78,589,763","78,743,364"
...,...,...,...,...,...
58,Note: The estimates are developed from a base ...,NaN,NaN,NaN,NaN
59,Suggested Citation:,NaN,NaN,NaN,NaN
60,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN
61,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN


In [17]:
# perform EDA Here as did on 2010s census data
census_20.drop(columns='Unnamed: 1', inplace=True)

In [18]:
census_20.rename(columns={'Unnamed: 0':'Geographic Area'},inplace=True)

In [19]:
census_20.dropna(inplace=True)

In [20]:
census_20=census_20.apply(lambda col: col.astype(str).str.replace(',', '').apply(pd.to_numeric, errors='ignore'))
census_20

,Geographic Area,2020,2021,2022
0,United States,331511512,332031554,333287557
1,Northeast,57448898,57259257,57040406
2,Midwest,68961043,68836505,68787595
3,South,126450613,127346029,128716192
4,West,78650958,78589763,78743364
...,...,...,...,...
52,.Washington,7724031,7740745,7785786
53,.West Virginia,1791420,1785526,1775156
54,.Wisconsin,5896271,5880101,5892539
55,.Wyoming,577605,579483,581381




# Join Data (Merge DataFrames)


Time to `merge`! Here I use the DataFrame method `df1.merge(right=df2, ...)` on DataFrame `df1` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)). Contrast this with the function `pd.merge(left=df1, right=df2, ...)` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html?highlight=pandas%20merge#pandas.merge)). Feel free to use either.

In [21]:
# Remove leading dots and strip whitespace
census['Geographic Area'] = census['Geographic Area'].str.lstrip('.').str.strip()
census_20['Geographic Area'] = census_20['Geographic Area'].str.lstrip('.').str.strip()

In [22]:
# First merge with census_2010
merged_2010 = tuberculosis_df.merge(
    right=census,
    left_on='U.S. jurisdiction',
    right_on='Geographic Area',
    how='inner'
)
merged_2010


,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Alabama,87,72,92,1.77,1.43,1.83,Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
1,Alaska,58,58,58,7.91,7.92,7.92,Alaska,710231,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
2,Arizona,183,136,129,2.51,1.89,1.77,Arizona,6392017,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
3,Arkansas,64,59,69,2.12,1.96,2.28,Arkansas,2915918,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
4,California,2111,1706,1750,5.35,4.32,4.46,California,37253956,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Virginia,191,169,161,2.23,1.96,1.86,Virginia,8001024,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519
47,Washington,221,163,199,2.90,2.11,2.57,Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893
48,West Virginia,9,13,7,0.50,0.73,0.39,West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147
49,Wisconsin,51,35,66,0.88,0.59,1.12,Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434


In [23]:
# Now merge the result with census_2020
tb_census_df = merged_2010.merge(
    right=census_20,
    left_on='U.S. jurisdiction',
    right_on='Geographic Area',
    how='inner',
    suffixes=('_x', '_y')
)
tb_census_df.drop(columns='Census',inplace=True)

In [24]:
# merge TB dataframe with two US census dataframes
tb_census_df

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area_x,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Geographic Area_y,2020,2021,2022
0,Alabama,87,72,92,1.77,1.43,1.83,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,Alabama,5031362,5049846,5074296
1,Alaska,58,58,58,7.91,7.92,7.92,Alaska,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,Alaska,732923,734182,733583
2,Arizona,183,136,129,2.51,1.89,1.77,Arizona,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,Arizona,7179943,7264877,7359197
3,Arkansas,64,59,69,2.12,1.96,2.28,Arkansas,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,Arkansas,3014195,3028122,3045637
4,California,2111,1706,1750,5.35,4.32,4.46,California,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,California,39501653,39142991,39029342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Virginia,191,169,161,2.23,1.96,1.86,Virginia,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519,Virginia,8636471,8657365,8683619
47,Washington,221,163,199,2.90,2.11,2.57,Washington,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893,Washington,7724031,7740745,7785786
48,West Virginia,9,13,7,0.50,0.73,0.39,West Virginia,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147,West Virginia,1791420,1785526,1775156
49,Wisconsin,51,35,66,0.88,0.59,1.12,Wisconsin,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434,Wisconsin,5896271,5880101,5892539


This is a little unwieldy. We could either drop the unneeded columns now, or just merge on smaller census DataFrames. Let's do the latter.


## Reproduce incidence

Let's recompute incidence to make sure we know where the original CDC numbers came from.

From the [CDC report](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down): TB incidence is computed as “Cases per 100,000 persons using mid-year population estimates from the U.S. Census Bureau.”

If we define a group as 100,000 people, then we can compute the TB incidence for a given state population as

$$\text{TB incidence} = \frac{\text{# TB cases in population}}{\text{# groups in population}} = \frac{\text{# TB cases in population}}{\text{population}/100000} $$

$$= \frac{\text{# TB cases in population}}{\text{population}} \times 100000$$

Let's try this for 2019:

In [25]:
tb_census_df["recompute incidence 2019"] = tb_census_df["TB cases 2019"]/tb_census_df["2019"]*100000
tb_census_df

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area_x,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Geographic Area_y,2020,2021,2022,recompute incidence 2019
0,Alabama,87,72,92,1.77,1.43,1.83,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,Alabama,5031362,5049846,5074296,1.77
1,Alaska,58,58,58,7.91,7.92,7.92,Alaska,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,Alaska,732923,734182,733583,7.93
2,Arizona,183,136,129,2.51,1.89,1.77,Arizona,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,Arizona,7179943,7264877,7359197,2.51
3,Arkansas,64,59,69,2.12,1.96,2.28,Arkansas,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,Arkansas,3014195,3028122,3045637,2.12
4,California,2111,1706,1750,5.35,4.32,4.46,California,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,California,39501653,39142991,39029342,5.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Virginia,191,169,161,2.23,1.96,1.86,Virginia,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519,Virginia,8636471,8657365,8683619,2.24
47,Washington,221,163,199,2.90,2.11,2.57,Washington,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893,Washington,7724031,7740745,7785786,2.90
48,West Virginia,9,13,7,0.50,0.73,0.39,West Virginia,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147,West Virginia,1791420,1785526,1775156,0.50
49,Wisconsin,51,35,66,0.88,0.59,1.12,Wisconsin,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434,Wisconsin,5896271,5880101,5892539,0.88


Awesome!!!

Let's use a for-loop and Python format strings to compute TB incidence for all years. Python f-strings are just used for the purposes of this demo, but they're handy to know when you explore data beyond this course ([Python documentation](https://docs.python.org/3/tutorial/inputoutput.html)).

In [26]:
# Define the years for which we want to compute the TB incidence
years = [2019, 2020, 2021]


for year in years:
    column_name = f"{year}"
    tb_cases_column = f"TB cases {year}"
    incidence_column_name = f"recompute incidence {year}"
    
    # Compute TB incidence and store it in a new column
    tb_census_df[incidence_column_name] = (
        tb_census_df[tb_cases_column] / tb_census_df[column_name] * 100000
    )


In [27]:
# recompute incidence for all years (2019, 2020, 2021)
tb_census_df

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area_x,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Geographic Area_y,2020,2021,2022,recompute incidence 2019,recompute incidence 2020,recompute incidence 2021
0,Alabama,87,72,92,1.77,1.43,1.83,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,Alabama,5031362,5049846,5074296,1.77,1.43,1.82
1,Alaska,58,58,58,7.91,7.92,7.92,Alaska,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,Alaska,732923,734182,733583,7.93,7.91,7.90
2,Arizona,183,136,129,2.51,1.89,1.77,Arizona,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,Arizona,7179943,7264877,7359197,2.51,1.89,1.78
3,Arkansas,64,59,69,2.12,1.96,2.28,Arkansas,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,Arkansas,3014195,3028122,3045637,2.12,1.96,2.28
4,California,2111,1706,1750,5.35,4.32,4.46,California,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,California,39501653,39142991,39029342,5.34,4.32,4.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Virginia,191,169,161,2.23,1.96,1.86,Virginia,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519,Virginia,8636471,8657365,8683619,2.24,1.96,1.86
47,Washington,221,163,199,2.90,2.11,2.57,Washington,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893,Washington,7724031,7740745,7785786,2.90,2.11,2.57
48,West Virginia,9,13,7,0.50,0.73,0.39,West Virginia,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147,West Virginia,1791420,1785526,1775156,0.50,0.73,0.39
49,Wisconsin,51,35,66,0.88,0.59,1.12,Wisconsin,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434,Wisconsin,5896271,5880101,5892539,0.88,0.59,1.12


These numbers look pretty close!!! There are a few errors in the hundredths place, particularly in 2021. It may be useful to further explore reasons behind this discrepancy. We'll leave it to you!

In [28]:
tb_census_df.describe()

,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,recompute incidence 2019,recompute incidence 2020,recompute incidence 2021
count,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00
mean,174.51,140.65,154.12,2.10,1.78,1.97,6065130.71,6108958.31,6153548.82,6195955.20,6241196.24,6286963.98,6332182.57,6372265.47,6405637.27,6436069.08,6500225.73,6510422.63,6535050.14,2.10,1.78,1.97
std,341.74,271.06,286.78,1.50,1.34,1.48,6838021.59,6901809.62,6967072.87,7029240.83,7100271.24,7172361.33,7235903.90,7288669.59,7327257.81,7360660.47,7408168.46,7394300.08,7423508.19,1.50,1.34,1.47
min,1.00,0.00,2.00,0.17,0.00,0.21,564487.00,567299.00,576305.00,582122.00,582531.00,585613.00,584215.00,578931.00,577601.00,578759.00,577605.00,579483.00,581381.00,0.17,0.00,0.21
25%,25.50,29.00,23.00,1.29,1.21,1.23,1700144.00,1712291.00,1724313.50,1732560.00,1740300.50,1746554.50,1756701.50,1767359.50,1777413.50,1789606.00,1820311.00,1844920.00,1857094.50,1.30,1.21,1.23
50%,70.00,67.00,69.00,1.80,1.52,1.70,4348181.00,4369821.00,4386346.00,4404659.00,4414349.00,4425976.00,4438182.00,4452268.00,4461153.00,4467673.00,4507445.00,4506589.00,4512310.00,1.81,1.52,1.69
75%,180.50,139.00,150.00,2.58,1.99,2.22,6654568.50,6720105.00,6780031.50,6838650.00,6908625.50,6996666.50,7117921.50,7233685.00,7340946.50,7446805.00,7451987.00,7502811.00,7572491.50,2.58,1.99,2.22
max,2111.00,1706.00,1750.00,7.91,7.92,7.92,37319502.00,37638369.00,37948800.00,38260787.00,38596972.00,38918045.00,39167117.00,39358497.00,39461588.00,39512223.00,39501653.00,39142991.00,39029342.00,7.93,7.91,7.90
